In [1]:
! pip install transformers -q
! pip install gdown
! sudo apt-get install unzip
! sudo apt-get install zip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-25ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 73 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
zip is already the newest version (3.0-11build1).
0 upgraded, 0 newly installed, 0 to remove and 73 not upgraded.


In [2]:
from PIL import Image
import requests
from transformers import AutoProcessor, CLIPVisionModel
import json
import os
import re
import torch
from torch.utils.data import DataLoader
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# upload and unzip 150k images in the temp storage
#! gdown 1iI6vzBPZ2GTs_-HZ9KAPHgrT_l0rJOsp
#! unzip -qq 'instruct150K.zip'

In [4]:
# Download clip model and processor from HG
model = CLIPVisionModel.from_pretrained("openai/clip-vit-large-patch14")
processor = AutoProcessor.from_pretrained("openai/clip-vit-large-patch14")

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.


In [21]:
model.to('cuda').eval()

CLIPVisionModel(
  (vision_model): CLIPVisionTransformer(
    (embeddings): CLIPVisionEmbeddings(
      (patch_embedding): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
      (position_embedding): Embedding(257, 1024)
    )
    (pre_layrnorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-23): 24 x CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (layer_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=1024, out_features=4096, bias=

In [25]:
image_dir = 'instruct150K'
batch_size = 32
imagefilenames = [f for f in os.listdir(image_dir) if re.match(r'[0-9a-zA-Z]+.*\.jpg', f)]
img_dataloader = DataLoader(imagefilenames, batch_size=batch_size,shuffle=False)

In [26]:
# process in chunks
for dl_idx,img_batch in enumerate(img_dataloader):
  images_batch = []
  for img in img_batch:
    image = Image.open(image_dir + '/' + img)
    images_batch.append(image)
      
  inputs = processor(images=images_batch, return_tensors="pt").to('cuda')
  outputs = model(**inputs)
  last_hidden_state = outputs.last_hidden_state.to('cpu')
  #print(last_hidden_state.shape)
    
  for i, label in enumerate(img_batch):
    #print(label)
    #print(last_hidden_state[i].shape)
    img_embed = last_hidden_state[i].clone().detach()
    torch.save(img_embed, './img_clip_embeddings/'+ label.split('.jpg')[0] + '.pt')
  print(f"Completed processing of {batch_size * (dl_idx + 1)} images") 

Completed processing of 32 images
Completed processing of 64 images
Completed processing of 96 images
Completed processing of 128 images
Completed processing of 160 images
Completed processing of 192 images
Completed processing of 224 images
Completed processing of 256 images
Completed processing of 288 images
Completed processing of 320 images
Completed processing of 352 images
Completed processing of 384 images
Completed processing of 416 images
Completed processing of 448 images
Completed processing of 480 images
Completed processing of 512 images
Completed processing of 544 images
Completed processing of 576 images
Completed processing of 608 images
Completed processing of 640 images
Completed processing of 672 images
Completed processing of 704 images
Completed processing of 736 images
Completed processing of 768 images
Completed processing of 800 images
Completed processing of 832 images
Completed processing of 864 images
Completed processing of 896 images
Completed processing of

In [27]:
! ls './img_clip_embeddings/' | wc -l

81479


In [ ]:
# zip up the directory
! zip -r instruct150kclipembed.zip img_clip_embeddings

  adding: img_clip_embeddings/ (stored 0%)
  adding: img_clip_embeddings/000000061834.pt (deflated 7%)
  adding: img_clip_embeddings/000000415378.pt (deflated 7%)
  adding: img_clip_embeddings/000000529248.pt (deflated 7%)
  adding: img_clip_embeddings/000000482740.pt (deflated 7%)
  adding: img_clip_embeddings/000000184435.pt (deflated 7%)
  adding: img_clip_embeddings/000000533228.pt (deflated 8%)
  adding: img_clip_embeddings/000000475980.pt (deflated 7%)
  adding: img_clip_embeddings/000000027158.pt (deflated 7%)
  adding: img_clip_embeddings/000000401083.pt (deflated 7%)
  adding: img_clip_embeddings/000000393428.pt (deflated 7%)
  adding: img_clip_embeddings/000000306407.pt (deflated 7%)
  adding: img_clip_embeddings/000000221865.pt (deflated 8%)
  adding: img_clip_embeddings/000000314298.pt (deflated 7%)
  adding: img_clip_embeddings/000000292312.pt (deflated 7%)
  adding: img_clip_embeddings/000000574236.pt (deflated 7%)
  adding: img_clip_embeddings/000000494424.pt (deflated 7

In [ ]:
# Read the json file
# !wget "https://huggingface.co/datasets/liuhaotian/LLaVA-Instruct-150K/blob/main/llava_instruct_150k.json"


# # get image filenames
# import json
# json_file_path = '/content/llava_instruct_150k.json'

# with open(json_file_path, 'r') as j:
#      contents = json.loads(j.read())
#!wget "http://images.cocodataset.org/val2017/000000039769.jpg"